In [11]:
import pandas as pd

# docs = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))['data']
# print(len(docs))
data_1 = pd.read_csv('./data_test/vnexpress-thoisu.csv')

data_2 = pd.read_csv('./data_test/vnexpress-thoisu-20.csv')

papers = pd.concat([data_1, data_2], ignore_index=True)
papers = papers.drop(columns=['description'], axis=1)
# papers.rename(columns={'body': 'paper_text'}, inplace=True)
# convert type of column
papers['body'].astype(str)
# Print head
print(papers.head())
print(papers.shape)

                                               title  \
0  Cao tốc Cần Thơ - Cà Mau có thêm nửa triệu m3 ...   
1  Thủ tướng trao Huân chương Hữu nghị cho bà Aur...   
2  Phà chở 15.000 lượt người mỗi ngày ở miền Tây ...   
3  Giám đốc Công an Hà Nội: Loại nhà nào cũng phả...   
4          Hàng loạt cây rừng bị đầu độc - VnExpress   

                                                body  \
0  Đây là tỉnh đầu tiên bàn giao mỏ cát cho nhà t...   
1  Lãnh đạo Chính phủ Việt Nam trao huân chương c...   
2  Công văn yêu cầu chấm dứt hoạt động bến phà An...   
3  Chiều 20/9, tại cuộc tiếp xúc cử tri quận Tây ...   
4  Ban quản lý rừng phòng hộ đầu nguồn liên hồ Sô...   

                                  date  
0   Thứ tư, 20/9/2023, 18:49 (GMT+7)\n  
1  Thứ năm, 21/9/2023, 01:29 (GMT+7)\n  
2  Thứ năm, 21/9/2023, 00:00 (GMT+7)\n  
3   Thứ tư, 20/9/2023, 22:11 (GMT+7)\n  
4   Thứ tư, 20/9/2023, 19:04 (GMT+7)\n  
(1222, 3)


In [12]:
import numpy as np
# duplicated
print(papers.duplicated().sum())
papers.drop_duplicates(inplace=True)

# drop nan
papers.dropna(axis=0, inplace=True, how="any")
# drop inf
papers.replace([np.inf, -np.inf], np.nan, inplace=True)
papers.replace('', np.nan, inplace=True)
papers.dropna(inplace=True,how='any',axis=0)
# Remove the columns
try:
    papers = papers.drop(columns=['id', 'event_type', 'pdf_name'], axis=1).sample(100)
except:
    pass

# Print out the first rows of papers
print(papers.head())
print(papers.info())
papers.reset_index(inplace=True, drop=True)

564
                                               title  \
0  Cao tốc Cần Thơ - Cà Mau có thêm nửa triệu m3 ...   
1  Thủ tướng trao Huân chương Hữu nghị cho bà Aur...   
2  Phà chở 15.000 lượt người mỗi ngày ở miền Tây ...   
3  Giám đốc Công an Hà Nội: Loại nhà nào cũng phả...   
4          Hàng loạt cây rừng bị đầu độc - VnExpress   

                                                body  \
0  Đây là tỉnh đầu tiên bàn giao mỏ cát cho nhà t...   
1  Lãnh đạo Chính phủ Việt Nam trao huân chương c...   
2  Công văn yêu cầu chấm dứt hoạt động bến phà An...   
3  Chiều 20/9, tại cuộc tiếp xúc cử tri quận Tây ...   
4  Ban quản lý rừng phòng hộ đầu nguồn liên hồ Sô...   

                                  date  
0   Thứ tư, 20/9/2023, 18:49 (GMT+7)\n  
1  Thứ năm, 21/9/2023, 01:29 (GMT+7)\n  
2  Thứ năm, 21/9/2023, 00:00 (GMT+7)\n  
3   Thứ tư, 20/9/2023, 22:11 (GMT+7)\n  
4   Thứ tư, 20/9/2023, 19:04 (GMT+7)\n  
<class 'pandas.core.frame.DataFrame'>
Index: 603 entries, 0 to 1216
Data col

In [13]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from underthesea import word_tokenize, sent_tokenize
from underthesea import ner
import torch

tokenizer = AutoTokenizer.from_pretrained("NlpHUST/ner-vietnamese-electra-base")
model = AutoModelForTokenClassification.from_pretrained("NlpHUST/ner-vietnamese-electra-base")

nlp = pipeline("token-classification", model=model, tokenizer=tokenizer)

def ner_results(text) -> dict[str, list[str]]: # dict[str, list[str]]: key là tên thực thể, value là list các từ thuộc thực thể đó
    sentences = sent_tokenize(text)
    # print(sentences)
    sentences = [s for s in sentences if len(s) > 0]

    list_ner = []
    results_sentences = []

    for s in sentences:
        if len(s) > 0:
            tensor_input = torch.tensor([tokenizer.encode(s)])
            # print(tensor_input.shape)
            if(tensor_input.shape[1] < 512):
                # print(preprocess(s))
                # print(nlp(s))
                if len(nlp(s)) > 0:
                    word = ""
                    original_word = ""
                    # s = preprocess(s)
                    for i in nlp(s):
                        if 'B-' in i['entity']:
                            word = i['word']
                            original_word = i['word']
                        elif 'I-' in i['entity']:
                            original_word += " " + i['word']
                            if i['word'] != "-":
                                word += " " + i['word']
                        elif 'E-' in i['entity']:
                            word += " " + i['word']
                            original_word += " " + i['word']
                            list_ner.append({'entity': i['entity'].split('-')[1], 'word': word})
                            word = ""
                            original_word = ""
                        elif 'S-' in i['entity']:
                            list_ner.append({'entity': i['entity'].split('-')[1], 'word': i['word']})
                            word = ""
                            original_word = ""
                    if word != "":
                        list_ner.append({'entity': i['entity'].split('-')[1], 'word': word})
                        # print(word)
                        s = s.replace(original_word, "_".join(word.split(" ")).lower())
                        # print(s)
                        results_sentences.append(s)
                        word = ""
                        original_word = ""
                #
                list_ner += nlp(s)
                # print(nlp(preprocess(s)))
                # print("=========================================")
            else:
                print("sentence too long")

    # print(list_ner)
    dict_ner = {}
    for i in list_ner:
        if i['entity'] in dict_ner:
            dict_ner[i['entity']].append(i['word'])
        else:
            dict_ner[i['entity']] = [i['word']]

    results = {}
    for key, value in dict_ner.items():
        if '-' not in key:
            # print(key + ": " + str(value))
            results[key] = value
    
    print(results_sentences)
    return results, " ".join(results_sentences)

ner_results_location_ = []
ner_results_person_ = []
ner_results_organization_ = []
doc = []
for i in range(len(papers['body'])):
    results, doc_ = ner_results(papers['body'][i])
    dict_ner = results
    doc.append(doc_)
    print(i)
    if 'LOCATION' not in dict_ner:
        dict_ner['LOCATION'] = []
    if 'PERSON' not in dict_ner:
        dict_ner['PERSON'] = []
    if 'ORGANIZATION' not in dict_ner:
        dict_ner['ORGANIZATION'] = []    
    ner_results_location_.append(dict_ner['LOCATION'])
    ner_results_person_.append(dict_ner['PERSON'])
    ner_results_organization_.append(dict_ner['ORGANIZATION'])
    print(dict_ner)
    
papers['location'] = ner_results_location_
papers['person'] = ner_results_person_
papers['organization'] = ner_results_organization_
papers['doc_ner'] = doc
papers.to_csv('./data_test/vnexpress-thoisu-ner1.csv', index=False)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


['Mỏ cát được giao nằm trên sông Tiền thuộc xã An Nhơn, huyện_châu_thành.', 'Đơn vị tiếp nhận là Tổng Công ty xây dựng số 1 - CTCP - nhà thầu thi công dự án thành phần đoạn Cần Thơ - Hậu Giang và Hậu Giang - Cà Mau, thuộc dự án cao_tốc_bắc_nam_phía_đông.', 'Đồng Tháp được giao cung ứng 10,3 triệu m3 trong năm nay và 2024 cho dự án cao_tốc_cần_thơ_cà_mau, trên tổng số 18,1 triệu m3 công trình này cần.', 'Số cát còn lại Chính phủ giao An Giang và vĩnh_long chịu trách nhiệm cung ứng, song hai địa phương này chưa bàn giao.', 'cao_tốc_cần_thơ_cà_mau dài 110 km, các tuyến nối khoảng 25 km, rộng 17 m, 4 làn, tổng vốn đầu tư hơn 27.200 tỷ đồng khởi công tháng 1/2023.', 'Công trình được xem là tuyến quan trọng nhất đồng_bằng_sông_cửu_long khi kết nối nhiều tỉnh thành với 128 cây cầu.', 'ngọc_tài']
0
{'LOCATION': ['huyện Châu Thành', 'cao tốc Bắc Nam phía Đông', 'cao tốc Cần Thơ Cà Mau', 'Vĩnh Long', 'Cao tốc Cần Thơ Cà Mau', 'Đồng bằng sông Cửu Long'], 'PERSON': ['Ngọc Tài'], 'ORGANIZATION': []